In [2]:
from compute import Var, Par
from modeling.api import Model, adda, addsolver
import sympy as sp
import numpy as np

In [30]:
import numpy as np

def PFi(idx, *args):
    Aij = np.array(((0, 0.3970, 0.8152, 0.9230, 0.1108),
                 (0.4252, 0, 0.6357, 0.7435, 0.1138),
                 (0.0329, 0.8856, 0, 0.3657, 0.0019),
                 (0.0878, 0.7248, 0.1978, 0, 0.0169),
                 (0.8955, 0.4568, 0.8075, 0.9239, 0)))
    Aoff = {
        'theta': (1, np.array((0.3,-0.3,-0.3,-0.2)), 
            np.array((0.4,-0.4,-0.4,0))),
        'Fo1': (1,6.25,0),
        'sigma1': (1, np.array((-0.75,0.5,-0.75,0.5)), 
            np.array((-2.5,0,-2.5,0,0))),
        'sigma2': (1, np.array((-0.5,0.333,-0.5,0.333,0.333)), 
            np.array((-1.111,0,-1.111,0,0))),
        'sigma3': (1, np.array((-0.375,0.25,-0.375,0.25,0.25)), 
            np.array((-0.625,0,-0.625,0,0))),
        'sigma4': (1, np.array((-0.3,0.2,-0.3,0.2, 0.2)), 
            np.array((-0.4,0,-0.4,0,0))),
        'sigma5': (1, np.array((-0.25,0.1667,-0.25,0.1667, 0.1667)), 
            np.array((-0.2778, 0, -0.2778,0,0))),
        'Fo2': (1, np.array((0.2,0.2)), np.array((0,0))),
        'Fo3': (1, 0, 0.04),
        'dpdx': (1,0.2,0),
        'temp': (1, np.array((0.3,-0.3,0.3)), 
            np.array((0.4,-0.4,0.4)))
    }
    Nargs = len(args)
    Aoff_sel = Aoff[idx]
    npargs = np.array(args)
    Aii = np.diag(Aoff_sel[2]) if Nargs>1 else Aoff_sel[2]
    A = Aij[0:Nargs,0:Nargs] + Aii
    total = Aoff_sel[0]+np.dot(Aoff_sel[1],npargs)+0.5*npargs@A@npargs.T
    return total

In [32]:
PFi('theta',1.2,1,1,1)

3.83321

In [ ]:
model = Model(rootname='root')
m = model.root
idbyname = True
tc = Var('t_c', unit='')
Sref = Var(r'S_{ref}', unit='m^2')
AR = Var('AR', unit='')
l = Var(r'\lambda', unit='')
L = Var('L', unit='')
WT = Var('W_T', unit='N')
WW = Var('W_W', unit='N')
x = Var('x', unit='')
struct = addsolver(m, name='struct', idbyname=idbyname)
# external
WFO = Var(r'W_{FO}', unit='N')
WO = Var(r'W_O', unit='N')
WE = Var(r'W_E', unit='N')
NZ = Var(r'N_Z', unit='N')
t = adda(struct, 't', tc*Sref/sp.sqrt(Sref*AR))
b = adda(struct, 'b', sp.sqrt(Sref*AR/2))
R = adda(struct, 'R', (1+2*l)/(3*(1+l)))
adda(struct, '\\theta', lambda *args: PFi('theta', *args),
    (x, b, R, L))
Fo1 = adda(struct, 'Fo1', lambda *args: PFi('Fo1', *args), (x,))
WFW = adda(struct, 5*Sref/18*2/3*t)
WF = adda(struct, WFW+WFO)
WT = adda(struct, WO+WW+WF+WE)
WW = adda(struct, 'W_W', 0.0051*(WT*NZ)**0.557*Sref**0.649*
    AR**0.5*tc**-0.4*(1+L)**0.1*(0.1875*Sref)**0.1/sp.cos(L)*Fo1)